In [75]:
import sys
sys.path.append('..')

In [76]:
import pandas as pd
from src.sentiment_analysis_BERT import *

input_df = pd.read_csv('../data/processed/cleaned_vertuo_pop_comments.csv')

In [77]:
analyzer = BERTSentimentAnalyzer()
results_df = analyzer.analyze_dataframe(input_df)

Using device: cpu
Loading BERT model...


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded successfully
Analyzing 245 comments...


Analyzing sentiment: 100%|██████████| 16/16 [00:08<00:00,  1.95it/s]


In [78]:
print(
"""
Video Statistics Summary:
--------------------------------------------------
Total Views: 1,149,975
Total Likes: 3,653
Total Dislikes: 278（7%）
Total Comments: 446 (including replies)
--------------------------------------------------
""")


Video Statistics Summary:
--------------------------------------------------
Total Views: 1,149,975
Total Likes: 3,653
Total Dislikes: 278（7%）
Total Comments: 446 (including replies)
--------------------------------------------------



In [79]:
quick_sentiment_summary(results_df)


Sentiment Summary:
--------------------------------------------------
Sentiment       Count  %      Total Likes 
--------------------------------------------------
Negative       :  135 ( 55.1%)       1442 likes
Neutral        :   82 ( 33.5%)        575 likes
Positive       :   28 ( 11.4%)        159 likes
--------------------------------------------------

Overall Summary:
------------------------------
Negative:  55.1% ( 135 comments,   1442 likes)
Neutral:   33.5% (  82 comments,    575 likes)
Positive:  11.4% (  28 comments,    159 likes)
Total:    100.0% ( 245 comments,   2176 likes)

Engagement Insights:
Negative comments received 66.3% of total likes
Positive comments received 7.3% of total likes

Average likes per comment:
Negative: 10.7 likes/comment
Neutral:  7.0 likes/comment
Positive: 5.7 likes/comment

Risk Assessment:
⚠️  HIGH NEGATIVE SENTIMENT - Requires immediate attention


In [80]:
validator = SentimentValidator(results_df)

In [83]:
validation_report = validator.full_validation_report()

SENTIMENT ANALYSIS VALIDATION REPORT

Statistical Validation:
------------------------------
Sentiment Distribution:
Negative       :  55.1%
Neutral        :  33.5%
Positive       :  11.4%

Likes by Sentiment:
                 count   sum   mean
sentiment_label                    
Negative           135  1442  10.68
Neutral             82   575   7.01
Positive            28   159   5.68

Statistical Warnings:
⚠️  Negative comments receiving high engagement - major issue

Confidence Distribution Analysis:
----------------------------------------
Very Low (<0.5):   10 (  4.1%)
Low (0.5-0.7):    56 ( 22.9%)
Medium (0.7-0.9): 103 ( 42.0%)
High (>0.9):       76 ( 31.0%)

Confidence Statistics:
Mean: 0.790
Std:  0.150
Min:  0.409
Max:  0.986

MANUAL VALIDATION SAMPLE
Manual validation sample saved to manual_validation_sample.csv
Instructions: Fill 'manual_label' column with your assessment
Mark 'correct' as True/False, add 'notes' if needed


In [82]:
validator.analyze_confidence_distribution()


Confidence Distribution Analysis:
----------------------------------------
Very Low (<0.5):   10 (  4.1%)
Low (0.5-0.7):    56 ( 22.9%)
Medium (0.7-0.9): 103 ( 42.0%)
High (>0.9):       76 ( 31.0%)

Confidence Statistics:
Mean: 0.790
Std:  0.150
Min:  0.409
Max:  0.986


{'statistics': count    245.000000
 mean       0.789925
 std        0.149896
 min        0.409208
 25%        0.668012
 50%        0.829471
 75%        0.914963
 max        0.985745
 Name: sentiment_confidence, dtype: float64,
 'distribution': {'very_low': 10, 'low': 56, 'medium': 103, 'high': 76},
 'warnings': [],
 'low_confidence_samples':                                                   text  ... sentiment_confidence
 87   What's the problem with having more buttons???...  ...             0.409208
 88   Just got mine out of the box and prepared a co...  ...             0.415892
 210  Just got mine out of the box but I'm thinking ...  ...             0.427236
 215                               This is so complex !  ...             0.442402
 167  Thanks to tik tok i got it ! This video is so ...  ...             0.453931
 1    Instructions from me to Nespresso, add a freak...  ...             0.477913
 56   The puzzles on Hogwarts Legacy were easier to ...  ...             0.486600
 